Permutation Test

Task 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
gene_1_type_1 = np.array([230, -1350, -1580, -400, -760])
gene_1_type_2 = np.array([970, 110, -50, -190, -200])
gene_2_type_1 = np.array([470, -850, -0.8, -280, 120])
gene_2_type_2 = np.array([390, -1730, -1360, -1, -330])

In [3]:
v1 = np.median(gene_1_type_1) # v1: median of gene 1 of Type I
v2 = np.median(gene_1_type_2) # v2: median of gene 1 of Type II

In [4]:
T_obs = np.abs(v1 - v2) # absolute difference between sample medians

In [5]:
# permutation test that shuffles the array and calculates T_perm for 1000 permutations

n_permutations = 1000
T_perms = []

gene_1 = np.concatenate([gene_1_type_1, gene_1_type_2])

for _ in range(n_permutations):
    np.random.shuffle(gene_1)

    type_1_perm = gene_1[0:5]
    type_2_perm = gene_1[5:10]

    v1_perm = np.median(type_1_perm)
    v2_perm = np.median(type_2_perm)

    T_perm = np.abs(v1_perm - v2_perm)
    T_perms.append(T_perm)



In [6]:
def find_p_value(T_perms, T_obs):
    n_perm = len(T_perms)
    perm_bigger_than_obs = 0

    for T_perm in T_perms:
        if T_perm > T_obs:
            perm_bigger_than_obs += 1
    
    p_value = (1 / n_perm) * perm_bigger_than_obs
    return p_value


In [7]:
gene_1 = np.concatenate([gene_1_type_1, gene_1_type_2])

p_value = find_p_value(T_perms, T_obs)
p_value

0.034

Task 2

In [8]:
from Bio import SeqIO
import pandas as pd

In [9]:
def calculate_gc_content(seq): # return GC content of a given sequence
    gc_count = seq.count("G") + seq.count("C")
    return gc_count / len(seq) if len(seq) > 0 else 0

In [10]:
def get_gc_contents(fasta_file): # returns a list of GC contents for all sequences in a fasta file
    gc_contents = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        gc_content = calculate_gc_content(str(record.seq).upper())
        gc_contents.append(gc_content)
    return gc_contents

In [11]:
gc_psittaci = get_gc_contents('chlamydia_psittaci.ffn')
gc_trachomatis = get_gc_contents('chlamydia_trachomatis.ffn')
gc_psittaci_median = np.median(gc_psittaci)
gc_trachomatis_median = np.median(gc_trachomatis)
print(f"Median of Psittaci GC contents: {gc_psittaci_median}")
print(f"Median of Trachomatis GC contents: {gc_trachomatis_median}")

Median of Psittaci GC contents: 0.3942060872753942
Median of Trachomatis GC contents: 0.4148590716818823


Permutation test the null hypothesis that the median of the GC content is 
the same for the two species

In [12]:
def shuffle_gc_contents(gc_species_1, gc_species_2): # shuffles the GC content between two species
    combined_gc = gc_species_1 + gc_species_2
    
    np.random.shuffle(combined_gc)
    
    # split the shuffled GC contents back into two species
    shuffled_species_1 = combined_gc[:len(gc_species_1)]
    shuffled_species_2 = combined_gc[len(gc_species_1): len(combined_gc)]
    
    return shuffled_species_1, shuffled_species_2

In [13]:
def calculate_permutations(n_permutations, gc_species_1, gc_species_2): # returns tuples of gc contents for species 1 and 2
    permutated_gc_content_medians = [] 
    for i in range(n_permutations):
        shuffled_gc_contents_1, shuffled_gc_contents_2 = shuffle_gc_contents(gc_species_1, gc_species_2)  # Shuffle the sequence
        permutated_gc_content_medians.append((np.median(shuffled_gc_contents_1), (np.median(shuffled_gc_contents_2))))
    return permutated_gc_content_medians

In [14]:
n_permutations = 1000

permutated_gc_content_medians = calculate_permutations(n_permutations, gc_psittaci, gc_trachomatis)
permutated_gc_content_medians[:5]

[(np.float64(0.4026974951830443), np.float64(0.40523363356664155)),
 (np.float64(0.4031141868512111), np.float64(0.40476190476190477)),
 (np.float64(0.4046511627906977), np.float64(0.4031443950129071)),
 (np.float64(0.40325670498084293), np.float64(0.40441731257287883)),
 (np.float64(0.4029535864978903), np.float64(0.4048689403429372))]

<small>Steps:

Calculate the median GC content for each species in each permutation and find the absolute difference. Save this absolute difference to T_perms for each permutation.

Calculate the p-value: The p-value is the proportion of permutated median differences that are greater than or equal to the observed difference.</small>

In [15]:
def get_differences_of_perms(perm_gc_content_medians):
    T_perms = []
    for gc_tuple in perm_gc_content_medians:
        T_perm = np.abs(gc_tuple[0] - gc_tuple[1])
        T_perms.append(T_perm)
    return T_perms

T_perms = get_differences_of_perms(permutated_gc_content_medians)
T_obs = np.abs(np.median(gc_psittaci) - np.median(gc_trachomatis))

# find p-value
p_value = find_p_value(T_perms, T_obs)
p_value

0.0

The p-value of 0 strongly suggests to reject the null hypothesis that the
median of the GC content is the same for the two species.

Biological Interpretation: 
The p-value of 0 suggests a significant difference in the median GC content between Chlamydia psittaci and Chlamydia trachomatis. This indicates that the two species have distinct genomic compositions, likely reflecting differences in their evolutionary adaptations.

Task 3

Try different sample sizes by subsampling the dataset. How does this change the permutation results?

In [16]:
import random

In [17]:
print(f"Size of GC Psitacci Array: {len(gc_psittaci)}")
print(f"Size of GC Trachomatis Array: {len(gc_trachomatis)}")

Size of GC Psitacci Array: 1119
Size of GC Trachomatis Array: 870


In [18]:
sample_sizes = [(len(gc_psittaci)/100, len(gc_trachomatis)/100),
                (len(gc_psittaci)/50, len(gc_trachomatis)/50),
                (len(gc_psittaci)/20, len(gc_trachomatis)/20),
                (len(gc_psittaci)/10, len(gc_trachomatis)/10),
                (len(gc_psittaci)/5, len(gc_trachomatis)/5)]

resampled_p_values = []

for sample_size in sample_sizes:    # ordered ascending
    sampled_gc_psittaci = random.sample(gc_psittaci, int(sample_size[0]))
    # print(sampled_gc_psittaci[:5])
    sampled_gc_trachomatis = random.sample(gc_trachomatis, int(sample_size[1]))
    # print(sampled_gc_trachomatis[:5])
    permutated_gc_content_medians_resample = calculate_permutations(n_permutations, sampled_gc_psittaci, sampled_gc_trachomatis)
    T_perms_resample = get_differences_of_perms(permutated_gc_content_medians)
    p_value = find_p_value(T_perms, T_obs)
    resampled_p_values.append(p_value)

print(resampled_p_values)

[0.0, 0.0, 0.0, 0.0, 0.0]


Task 4

Test for the difference in the maximum of the GC content instead of median

In [19]:
def calculate_permutations_maximum(n_permutations, gc_species_1, gc_species_2): # returns tuples of gc contents for species 1 and 2
    permutated_gc_content_maximums = [] 
    for i in range(n_permutations):
        shuffled_gc_contents_1, shuffled_gc_contents_2 = shuffle_gc_contents(gc_species_1, gc_species_2)  # Shuffle the sequence
        permutated_gc_content_maximums.append((np.max(shuffled_gc_contents_1), (np.max(shuffled_gc_contents_2))))
    return permutated_gc_content_medians

In [20]:
permutated_gc_content_maximums = calculate_permutations(n_permutations, gc_psittaci, gc_trachomatis)
permutated_gc_content_maximums[:5]

T_perms_max = get_differences_of_perms(permutated_gc_content_medians)
print(T_perms_max[:10])
T_obs_max = np.abs(np.max(gc_psittaci) - np.max(gc_trachomatis))
print(T_obs_max)

resampled_p_values_max = []

for sample_size in sample_sizes:    # ordered ascending
    sampled_gc_psittaci = random.sample(gc_psittaci, int(sample_size[0]))
    # print(sampled_gc_psittaci[:5])
    sampled_gc_trachomatis = random.sample(gc_trachomatis, int(sample_size[1]))
    # print(sampled_gc_trachomatis[:5])
    permutated_gc_content_medians_resample = calculate_permutations_maximum(n_permutations, sampled_gc_psittaci, sampled_gc_trachomatis)
    T_perms_resample = get_differences_of_perms(permutated_gc_content_medians)
    p_value = find_p_value(T_perms, T_obs)
    resampled_p_values_max.append(p_value)

print(resampled_p_values_max)

[np.float64(0.0025361383835972306), np.float64(0.0016477179106936801), np.float64(0.0015067677777905741), np.float64(0.0011606075920359005), np.float64(0.0019153538450468743), np.float64(0.0005850552095905126), np.float64(0.000610964438461592), np.float64(0.0014294805973180669), np.float64(0.0005798211010761278), np.float64(0.0008127744148703764)]
0.05830039525691699
[0.0, 0.0, 0.0, 0.0, 0.0]


In [21]:
# plot p-value relative to sample size